We will see how can we use builtin RNN module of PyTorch.
- Implement a Recurrent Neural Net (RNN) in PyTorch! Learn how we can use the `nn.RNN` module and work with an input sequence. I also show you how easily we can switch to a gated recurrent unit (GRU) or long short-term memory (LSTM) RNN.
- we will start with feedforward NN for digit classification of MNIST dataset.
- Image classificaion is not a typical example for RNN. But we will se how to treat the input as a sequence and have the correct shape.
- We can treat input as a sequence.
- In our example we used many to one example, where we ahve only one output from a sequenc of input.

https://github.com/patrickloeber/pytorch-examples/blob/master/rnn-lstm-gru/main.py

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
# input_size = 784 # 28x28
# hidden_size = 500 
# num_classes = 10
# num_epochs = 2
# batch_size = 100
# learning_rate = 0.001
# For RNN we treat our input as a sequence. So we treat one image dimension as one sequence and other image dimension as an input or feature size
input_size =28
sequence_length = 28
hidden_size = 128 
num_classes = 2 # by default it is one. But here we are stacking two RNNS together. 2nd RNN takes output from 1st RNN as an input. This will improve the model
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [ ]:

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

examples = iter(test_loader)
example_data, example_targets = next(examples)

for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(example_data[i][0], cmap='gray')
plt.show()


- We implement the RNN, change the NeuralNet to RNN.
- The RNN has the input size, hidden size, and num_classes as parameters. And there is new parameter  number of layers

In [ ]:
# Fully connected neural network with one hidden layer
# class NeuralNet(nn.Module):
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        # we must pass the parmateers to our model when we create. these are hyperparemet. 
        # We simply want to store the number of layers and hidden size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True) # batch as a first diementison
        #  x -> batch_size, seq, input_size
        # At end we have one output so we have one more layer with softmax. See from many to one example
        # we only used the last timestep of our sequence as classification
        self.fc = nn.Linear(hidden_size, num_classes)
        
        
    def forward(self, x):
        # intial hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), slef.hidden_size).to(device)
        # call our RNN model
        out, _ = self.rnn(x, h0)
        # out shape: batch_size, seq_length, hidden_size
        # out (N,28,128) decode to get only last timestep
        out = out[:,1-,:]
        # out (N, 128)


In [ ]:

model = RNN(input_size, hidden_size,  num_layers,  num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100,1, 28, 28]
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

# Test the model: Evalaution
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

# GRU: https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
LSTM https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

instead of ` nn.RNN()` we use ` nn.gru` ` nn.LSTM`where we inherit the class. It takes all other things same.

for LSTFM we will need intial tensor for the cell state. So modify forward
```
def forward(self, x):
        # intial hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), slef.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), slef.hidden_size).to(device)
        out, _ = self.lstm(x, (h0,c0))
        # out shape: batch_size, seq_length, hidden_size
        # out (N,28,128) decode to get only last timestep
        out = out[:,1-,:]
        # out (N, 128)
```